<a href="https://colab.research.google.com/github/andryll/MGR-IC/blob/main/codes/MGR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
!git clone https://github.com/andryll/MGR-IC.git

Cloning into 'MGR-IC'...
remote: Enumerating objects: 1044, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 1044 (delta 14), reused 8 (delta 0), pack-reused 999
Receiving objects: 100% (1044/1044), 1.13 GiB | 29.92 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (1004/1004), done.


In [66]:
def readSongs (genre, numSongs, sr):
  dir_path = os.path.join('/content/MGR-IC/songs/',genre)

  files = os.listdir(dir_path)
  files.sort

  songs = []

  for i in range(numSongs):
    songs.append(librosa.load(os.path.join(dir_path, files[i]), sr=sr, mono = True))
    songs[-1] = songs[-1] + (genre,)
 # print(songs)

  return songs


def featureExtraction (songs, feature, sr):

  zcrlist = []
  rmslist = []
  mfcclist = []
  sctoidlist = []
  srlofflist99 = []
  srlofflist1 = []

  for i in songs:

    match feature:
      case 'zcr':
        zcr = librosa.feature.zero_crossing_rate(i[0], frame_length = 2048, hop_length = 512, center = True)
        aux = pd.DataFrame(zcr[0]).transpose()
        aux['Classe'] = i[2]
        zcrlist.append(aux)

      case 'rms':
        rms = librosa.feature.rms(y=i[0], frame_length = 2048, hop_length = 512, center = True)
        aux = pd.DataFrame(rms[0]).transpose()
        aux['Classe'] = i[2]
        rmslist.append(aux)

      case 'mfcc':
        mfcc = librosa.feature.mfcc(y=i[0], sr=sr)
        aux = pd.DataFrame(mfcc).transpose()
        aux['Classe'] = i[2]
        mfcclist.append(aux)

      case 'centroid':
        sctoid = librosa.feature.spectral_centroid(y=i[0], sr=sr, hop_length = 512, center = True)
        aux = pd.DataFrame(sctoid[0]).transpose()
        aux['Classe'] = i[2]
        sctoidlist.append(aux)

      case 'rolloff':
        srloff99 = librosa.feature.spectral_rolloff(y=i[0], sr=sr, hop_length = 512, center = True, roll_percent=0.95)
        srloff1 = librosa.feature.spectral_rolloff(y=i[0], sr=sr, hop_length = 512, center = True, roll_percent=0.05)
        aux = pd.DataFrame(srloff99).transpose()
        aux['Classe'] = i[2]
        srlofflist99.append(aux)
        aux = pd.DataFrame(srloff1).transpose()
        aux['Classe'] = i[2]
        srlofflist1.append(aux)

      case 'all':
        zcr = librosa.feature.zero_crossing_rate(i[0], frame_length = 2048, hop_length = 512, center = True)
        aux = pd.DataFrame(zcr[0]).transpose()
        aux['Classe'] = i[2]
        zcrlist.append(aux)

        rms = librosa.feature.rms(y=i[0], frame_length = 2048, hop_length = 512, center = True)
        aux = pd.DataFrame(rms[0]).transpose()
        aux['Classe'] = i[2]
        rmslist.append(aux)

        mfcc = librosa.feature.mfcc(y=i[0], sr=sr)
        aux = pd.DataFrame(mfcc).transpose()
        aux['Classe'] = i[2]
        mfcclist.append(aux)

        sctoid = librosa.feature.spectral_centroid(y=i[0], sr=sr, hop_length = 512, center = True)
        aux = pd.DataFrame(sctoid[0]).transpose()
        aux['Classe'] = i[2]
        sctoidlist.append(aux)

        srloff99 = librosa.feature.spectral_rolloff(y=i[0], sr=sr, hop_length = 512, center = True, roll_percent=0.95)
        srloff1 = librosa.feature.spectral_rolloff(y=i[0], sr=sr, hop_length = 512, center = True, roll_percent=0.05)
        aux = pd.DataFrame(srloff99[0]).transpose()
        aux['Classe'] = i[2]
        srlofflist99.append(aux)
        aux = pd.DataFrame(srloff1[0]).transpose()
        aux['Classe'] = i[2]
        srlofflist1.append(aux)

      case _:
        print('Feature Inválida')

  match feature:
    case 'zcr':
      dfzcr = pd.concat(zcrlist, axis=0)
      return dfzcr

    case 'rms':
      dfrms = pd.concat(rmslist, axis=0)
      return dfrms

    case 'mfcc':
      dfmfcc = pd.concat(mfcclist, axis=0)
      return dfmfcc

    case 'centroid':
      dfcentroid = pd.concat(sctoidlist, axis=0)
      return dfcentroid

    case 'roloff':
      dfrolloff01 = pd.concat(sclofflist1, axis=0)
      dfrolloff99 = pd.concat(sclofflist99, axis=0)
      return [dfrolloff01, dfrolloff99]

    case 'all':
      dfzcr = pd.concat(zcrlist, axis=0)
      dfrms = pd.concat(rmslist, axis=0)
      dfmfcc = pd.concat(mfcclist, axis=0)
      dfcentroid = pd.concat(sctoidlist, axis=0)
      dfrolloff01 = pd.concat(sclofflist1, axis=0)
      dfrolloff99 = pd.concat(sclofflist99, axis=0)
      return dfzcr, dfrms, dfmfcc, dfcentroid, dfrolloff01, dfrolloff99



In [67]:
songs = readSongs ('rock', 10, 44100)
[df, df2] = featureExtraction (songs, 'rolloff', 44100)
df.head()
df2.head()

TypeError: ignored